In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Carrega variáveis de ambiente
load_dotenv()

def get_oracle_engine() -> create_engine:
    """Cria conexão com Oracle a partir das variáveis de ambiente."""
    user = os.getenv("ORACLE_USER")
    password = os.getenv("ORACLE_PASSWORD")
    host = os.getenv("ORACLE_HOST")
    port = os.getenv("ORACLE_PORT", "1521")
    service_name = os.getenv("ORACLE_SERVICE")
    
    if not all([user, password, host, service_name]):
        raise ValueError("Verifique as variáveis de ambiente do Oracle.")
    
    connection_string = f"oracle+oracledb://{user}:{password}@{host}:{port}/?service_name={service_name}"
    return create_engine(connection_string)

def get_sqlite_engine(db_path: str) -> create_engine:
    """Cria conexão com SQLite."""
    return create_engine(f"sqlite:///{db_path}")

def consultar_oracle(query: str, oracle_engine: create_engine) -> pd.DataFrame:
    """Executa query no Oracle e retorna DataFrame."""
    with oracle_engine.connect() as conn:
        df = pd.read_sql(query, con=conn)
    return df

def salvar_dados(df: pd.DataFrame, sqlite_engine: create_engine, nome_tabela: str, export_csv: bool = True):
    """Salva DataFrame em SQLite e opcionalmente em CSV."""
    with sqlite_engine.connect() as conn:
        df.to_sql(nome_tabela, con=conn, if_exists="replace", index=False)
    if export_csv:
        df.to_csv(f"data/{nome_tabela}.csv", index=False, encoding='utf-8-sig')

oracle_engine = get_oracle_engine()
sqlite_engine = get_sqlite_engine("data/dados.db")

In [4]:
query = """SELECT * FROM SIGEF_STAGE.STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA__RECEITA__DETALHE"""
df = consultar_oracle(query, oracle_engine)
print(df.head())  # Exibe as primeiras linhas do DataFrame


    sigef_db data_lancamento         id_unidade_gestora_gestao  \
0  SIGEF2024      2024-07-19  EB3D5A8F422494D32E19DAED25D59B0A   
1  SIGEF2024      2024-07-19  EB3D5A8F422494D32E19DAED25D59B0A   
2  SIGEF2024      2024-07-19  EB3D5A8F422494D32E19DAED25D59B0A   
3  SIGEF2024      2024-07-19  EB3D5A8F422494D32E19DAED25D59B0A   
4  SIGEF2024      2024-07-19  EB3D5A8F422494D32E19DAED25D59B0A   

                   id_fonte_recurso               id_natureza_receita  \
0  053F3FFFD52D585E61E90EC9C06D8964  74FDDBB1A7A1AB59DBD6334731DF7CE9   
1  053F3FFFD52D585E61E90EC9C06D8964  01ECC245F94A32FC8449F4B24D825F75   
2  053F3FFFD52D585E61E90EC9C06D8964  74FDDBB1A7A1AB59DBD6334731DF7CE9   
3  053F3FFFD52D585E61E90EC9C06D8964  3A66E1119B11543AB4071201EB9A758B   
4  053F3FFFD52D585E61E90EC9C06D8964  96076B1FCC2A290F247BD4455677EF1D   

                  id_conta_contabil                         id_evento  \
0  119155132F62BA6B82D88C464FF11214  393BA989502498A4C8A1EC0A960A3947   
1  119155132F62BA6

In [5]:
salvar_dados(df, sqlite_engine, nome_tabela="TESTE")
print("✅ Dados copiados com sucesso!")

✅ Dados copiados com sucesso!
